# Slit through focus analysis for SM slit alignment 

**Creator :** Fabrice MADEC *(Laboratoire d'astrophysique de Marseille)*

**Editor :** Romain LHOUSSAINE BEN BRAHIM *(Laboratoire d'astrophysique de Marseille)*

**Date :** April, 06 2021 ***[Closed on April, 7 2021]***

**Data required:** You need to use ics_spsaitActor to perform your sequence,get the data required acquisitions to plot the through focus from the slit, and being able to find the right focus.

    Example : slit throughfocus exptime=6.0 lowBound=-0.5 upBound=1.5 fiber=engbotend nbPosition=10 duplicate=1
    (lowBound and upBound are the ranges of the slit focus)

## 0. Librairies, constants and parameters

### 0.1. Classic librairies

In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import os
import datetime
from matplotlib import style
from matplotlib.offsetbox import AnchoredText

### 0.2. LAM Librairies

In [2]:
# import LAM library for logbook, data analysis...
from pfs.lam.sacFileHandling import Logbook, constructFilelist
import pfs.lam.imageAnalysis as imeas
from pfs.lam.slitFocusAnalysis import *
from pfs.lam.style import colors

ImportError: cannot import name 'register_translation' from 'skimage.feature' (/software/stack/20220609/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-3.0.0/lib/python3.8/site-packages/skimage/feature/__init__.py)

### 0.3. Constants & parameters

#### 0.3.1 Constants & Paths

In [ ]:
# Style sheets reference
style.use('ggplot')

In [ ]:
# Distance between engtopend & engbotend fibers
CST_DIST_FIBERS = 138.84

In [ ]:
smId = 2

In [ ]:
imgPath = f'/home/pfs/shared/Pictures/SM{smId}/slitAlign/' # WARNING : You must verify that you are choosing the right SM folder !!

#### 0.3.1 Parameters of the slit

In [ ]:
#All the experiment Ids used for SM3 slit Alignement

# First analysis on engineering fibers
#experimentIds = [479,480,481,482]     # Ids needed for the slit through focus analysis

#TO ADD ON CELL 24 :
#focusModel['fiber'] = focusModel['fiber'].str.strip("'")

# First analysis on engineering fibers
#experimentIds = [483,484,485,486]     # Ids needed for the slit through focus analysis

# Second analysis on engineering fibers : after the first correction
#experimentIds = [488,489,490,491]     # Ids needed for the slit through focus analysis


In [ ]:
# Experiment Ids used for evalutating the background
#bck_expId     = 260                  #
doBck         = True                  # If you don't made any background, you can set doBck as True to retrieve the median value of the image

In [ ]:
com           = True                  #
corrector     = False                 #
doSave        = True                  # Parameter that allow to save the through slit focus on the distant computer 
head          = 0                     #
tail          = 0                     #
dfs           = []                    # Dataframe containing all 


In [ ]:
threshold = 300                   #
radius = 60                       #
doPrint = False                   #
doPlot = False                    #
roi_size = 150                    #

In [ ]:
fwhm_radius = radius              #
fwhm_method = 'gaussian'          #

In [ ]:
vline = True
index = 'fca_x'

In [ ]:
thres_neighbor = -0.1             #
doNeighborFilter = True           #
plotModel = True

## 1. Downloading Data

### 1.0. Theorical Zeemax data

In [ ]:
# Filepath of data model
pmodel = os.environ['LAM_SPS_ANALYSIS_DIR']+"/notebooks/optical/input/slit"
file = 'slit-defParam-Ouverture100-fiber65.csv'

In [ ]:
# 1: Loading Zeemax data
zemaxData = pd.read_csv('%s/%s' % (pmodel, file), delimiter=" ")

In [ ]:
# 2: Loading Zeemax data for medium fibers
zemaxMidFit = imeas.fitparabola(x=zemaxData.Slitdefocus, y=zemaxData.MidFiber, deg=15, focus='max')

In [ ]:
# 3: Loading Zeemax data for extreme fibers
zemaxEndFit = imeas.fitparabola(x=zemaxData.Slitdefocus, y=zemaxData.ExtremeFiber, deg=15, focus='max')

### 1.1. Experimental data

In [ ]:
# Loading data from the data base
experiments = pd.read_sql_query('select * from Experiment where type="slitAlignment" order by experimentId desc',
                                con='sqlite:////data/ait/experimentLog-sac.db', index_col='experimentId')

In [ ]:
# Data manipulation to recover the fiber and exptime informations
experiments['exptime'] = [Logbook.getParameter(experimentId, 'exptime') for experimentId in experiments.index]
experiments['fiber'] = [Logbook.getParameter(experimentId, 'fiber', doRaise=False) for experimentId in experiments.index]

In [ ]:
# Display of the 4 acquisitions made on the SacSequence Panel
experiments[experiments.index.isin(experimentIds)]

## 2. Data Analysis

### 2.1. Compile the maximum brightness according to the focus's displacement

In [ ]:
for experimentId in experimentIds:
    dfs.append(getSlitTF(experimentId=experimentId, com=com, head=head, tail=tail, doBck=doBck, doPrint=doPrint, doPlot=doPlot))

cube = pd.concat(dfs)

### 2.2. Plot framed energy by 20/40 pixels over the optical axis

#### 2.2.1. Supressing aberrant values

In [ ]:
if doNeighborFilter :
    cube2 = imeas.neighbor_outlier_filter(cube, "EE20",thres_neighbor, absolute=True)

In [ ]:
thfocModel= fitFocusData(cube2[cube2.EE20_nbh_flag], corrector=corrector, doPlot=doPlot)

#### 2.2.3. Supressing aberrant values

In [ ]:
focusModel = getFocusModel(thfocModel)

## 3. Tilt & Focus determination

### 3.1. Tilt determination

In [ ]:
# 1 : Computing position difference between both extreme fibers
[tilt_ry] = focusModel.fca_x[focusModel.fiber.str.match('\w{3}botend')==True].values - focusModel.fca_x[focusModel.fiber.str.match('\w{3}topend')==True].values

In [ ]:
# 2 : Computing tilt on y axis (rad)
angle=np.arctan(tilt_ry/CST_DIST_FIBERS)

In [ ]:
# 3 : Converting in degrees
angle_degre=(angle*180)/(np.pi)

In [ ]:
# 4 : Displying the tilt value
print("There is a tilt of %.3f milidegrees" %(angle_degre*1000))

### 3.2. Focus determination

In [ ]:
# 1 : Calculating mean value between the both mid position fibers
[focus_adjustment] = ((focusModel.fca_x[focusModel.fiber.str.match('\w{3}botmid')==True].values + focusModel.fca_x[focusModel.fiber.str.match('\w{3}topmid')==True].values)/2) 

In [ ]:
# 2 : Calculating mean value between the both mid position fibers
[focus_adjustment] = [focus_adjustment] - zemaxMidFit.focus

In [ ]:
# 3 : Displying the defocus
print("You have to adjust the focus by %.3f microns" %(focus_adjustment*1000))

## 4. Display Slit Through Focus graph

### 4.1. Graph

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(111)
j=2

if plotModel :
    ax1.plot(zemaxData.Slitdefocus, zemaxData.MidFiber, '+', color=colors[0], label='Zemax_MidFiber = %.3f' % zemaxMidFit.focus)
    ax1.plot(zemaxMidFit.x, zemaxMidFit.y, '--', color=colors[0])
    if vline:
        ax1.vlines(color=colors[0], **zemaxMidFit.vline)

    ax1.plot(zemaxData.Slitdefocus, zemaxData.ExtremeFiber, '+', color=colors[1], label='Zemax_EndFiber = %.3f' % zemaxEndFit.focus)
    ax1.plot(zemaxEndFit.x, zemaxEndFit.y, '--', color=colors[1])
    if vline:
        ax1.vlines(color=colors[1], **zemaxEndFit.vline)
else :
    print("Zemax model for engineering fibers non plotted")


    
for experimentId, df in cube2[cube2.EE20_nbh_flag].groupby('experimentId'):
    fit = thfocModel.query("experimentId==%d"%(experimentId))
    focus = focusModel.query("experimentId==%d and EE20=='%s'"%(experimentId, "EE20"))

    ax1.plot(df[index], df["EE20"], 'o', color=colors[j], 
    label='expId%d:%s = %.3f' % (experimentId, experiments.fiber[experimentId], focus[index]))

    ax1.plot(fit[index], fit["EE20"], '--', color=colors[j])

    if vline:
        ax1.vlines(x=focus[index], ymin=fit["EE20"].min(), ymax = fit["EE20"].max(), color=colors[j])
        j+=1
    
lns = [line for i, line in enumerate(ax1.get_lines()) if not i % 2]
labs = [line.get_label() for line in lns]

at = AnchoredText(" Tilt-Y = %.3f millidegrees"%(angle_degre*1000),
                  prop=dict(size=15), frameon=True,
                  loc='upper left',
                  )
at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
ax1.add_artist(at)
at2 = AnchoredText(" Defocus = %.3f microns"%(focus_adjustment*1000),
                  prop=dict(size=15), frameon=True,
                  loc='lower left',
                  bbox_to_anchor=(0., 0.835),
                  bbox_transform=ax1.transAxes
                  )
at2.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")


ax1.add_artist(at2)

ax1.legend(lns, labs)
ax1.set_xlabel('%s(mm)'%index)
ax1.set_ylabel("EE20")

plt.title(f'SM{smId} Slit Through focus : Zemax vs {str(focusModel.fiber[0][:3])}_Fibers \n ExperimentIds = {experimentIds} \n Criteria : EE20 doBck={doBck} \n Date = {datetime.date.today()}  ')
plt.grid()
plt.show()



### 4.2. Saving graph

In [ ]:
if doSave : 
    fig.savefig(imgPath+f"Slit_Through_focus_SM{smId}_{focusModel.fiber[0][:3]}_Exp{experimentIds}_{datetime.date.today()}.png") 